In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

data = pd.read_csv('/content/drive/MyDrive/Kalla/aspal/Harga-minyak2.csv',parse_dates=['Start_date', 'End_date'])

In [3]:
data

,Week Of,Start_date,End_date,Mon,Tue,Wed,Thu,Fri,Oil_Low,Oil_Mid,Oil_High,Argus_Low,Argus_High,Argus_Mid
0,2016 Nov-21 to Nov-25,2016 Nov-21,2016 Nov-25,45.96,46.10,46.54,47.54,46.32,45.96,46.750,47.54,200.0,210.0,205.0
1,2016 Nov-28 to Dec- 2,2016 Nov-28,2016 Dec- 2,46.64,44.68,47.95,52.28,52.35,44.68,48.515,52.35,200.0,210.0,205.0
2,2016 Dec- 5 to Dec- 9,2016 Dec- 5,2016 Dec- 9,53.30,52.31,51.90,51.60,52.19,51.60,52.450,53.30,205.0,215.0,210.0
3,2016 Dec-12 to Dec-16,2016 Dec-12,2016 Dec-16,53.99,53.28,53.15,51.72,54.15,51.72,52.935,54.15,230.0,240.0,235.0
4,2016 Dec-19 to Dec-23,2016 Dec-19,2016 Dec-23,53.53,54.56,53.01,54.04,53.93,53.01,53.785,54.56,240.0,250.0,245.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,2023 Sep-11 to Sep-15,2023 Sep-11,2023 Sep-15,91.97,93.58,93.04,95.20,95.55,91.97,93.760,95.55,479.0,505.0,492.0
352,NaN,2023 Sep-18,2023 Sep-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484.0,502.0,493.0
353,NaN,2023 Sep-25,2023 Sep-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483.0,497.0,490.0
354,NaN,2023 Oct-2,2023 Oct-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,487.0,508.0,497.5


In [4]:
columns_to_clean = ['Start_date', 'End_date']

for column in columns_to_clean:
    data[column] = data[column].astype(str)  # Convert column to string data type
    data[column] = data[column].str.replace(r'\s+', '', regex=True)
    data[column] = data[column].str[:4] + '-' + data[column].str[4:7] + data[column].str[7:]

# Display the resulting DataFrame
print(data)


                     Week Of   Start_date     End_date   Mon    Tue    Wed   \
0      2016 Nov-21 to Nov-25  2016-Nov-21  2016-Nov-25  45.96  46.10  46.54   
1      2016 Nov-28 to Dec- 2  2016-Nov-28   2016-Dec-2  46.64  44.68  47.95   
2      2016 Dec- 5 to Dec- 9   2016-Dec-5   2016-Dec-9  53.30  52.31  51.90   
3      2016 Dec-12 to Dec-16  2016-Dec-12  2016-Dec-16  53.99  53.28  53.15   
4      2016 Dec-19 to Dec-23  2016-Dec-19  2016-Dec-23  53.53  54.56  53.01   
..                       ...          ...          ...    ...    ...    ...   
351    2023 Sep-11 to Sep-15  2023-Sep-11  2023-Sep-15  91.97  93.58  93.04   
352                      NaN  2023-Sep-18  2023-Sep-22    NaN    NaN    NaN   
353                      NaN  2023-Sep-25  2023-Sep-29    NaN    NaN    NaN   
354                      NaN   2023-Oct-2   2023-Oct-6    NaN    NaN    NaN   
355                      NaN   2023-Oct-9  2023-Oct-13    NaN    NaN    NaN   

      Thu    Fri   Oil_Low  Oil_Mid  Oil_High  Argu

In [5]:
data['Start_date'] = pd.to_datetime(data['Start_date'])
data['End_date'] = pd.to_datetime(data['End_date'])

In [6]:
print(data.isna().sum)

<bound method NDFrame._add_numeric_operations.<locals>.sum of      Week Of  Start_date  End_date   Mon    Tue    Wed    Thu    Fri   \
0      False       False     False  False  False  False  False  False   
1      False       False     False  False  False  False  False  False   
2      False       False     False  False  False  False  False  False   
3      False       False     False  False  False  False  False  False   
4      False       False     False  False  False  False  False  False   
..       ...         ...       ...    ...    ...    ...    ...    ...   
351    False       False     False  False  False  False  False  False   
352     True       False     False   True   True   True   True   True   
353     True       False     False   True   True   True   True   True   
354     True       False     False   True   True   True   True   True   
355     True       False     False   True   True   True   True   True   

     Oil_Low  Oil_Mid  Oil_High  Argus_Low  Argus_High  Argus

In [7]:
#menghapus kolom
df = data.drop(data.columns[[0, 3, 4, 5, 6, 7, 8, 9 ,10]], axis=1)

In [8]:
df


,Start_date,End_date,Argus_Low,Argus_High,Argus_Mid
0,2016-11-21,2016-11-25,200.0,210.0,205.0
1,2016-11-28,2016-12-02,200.0,210.0,205.0
2,2016-12-05,2016-12-09,205.0,215.0,210.0
3,2016-12-12,2016-12-16,230.0,240.0,235.0
4,2016-12-19,2016-12-23,240.0,250.0,245.0
...,...,...,...,...,...
351,2023-09-11,2023-09-15,479.0,505.0,492.0
352,2023-09-18,2023-09-22,484.0,502.0,493.0
353,2023-09-25,2023-09-29,483.0,497.0,490.0
354,2023-10-02,2023-10-06,487.0,508.0,497.5


In [9]:
#mengisi nilai NAN berdasarkan data sebelumnya
df = df.fillna(method='ffill')

In [10]:
print(df.isna().sum())

Start_date    0
End_date      0
Argus_Low     0
Argus_High    0
Argus_Mid     0
dtype: int64


In [11]:
argus = df[['Start_date','Argus_High']]
print("Shape of Argus_High dataframe:", argus.shape)

Shape of Argus_High dataframe: (356, 2)


In [12]:
copy_price = argus.copy()
del argus['Start_date']
scaler=MinMaxScaler(feature_range=(0,1))
argus=scaler.fit_transform(np.array(argus).reshape(-1,1))
print(argus.shape)

(356, 1)


In [13]:
training_size=int(len(argus)*0.65)
test_size=len(argus)-training_size
train_data,test_data=argus[0:training_size,:],argus[training_size:len(argus),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (231, 1)
test_data:  (125, 1)


In [14]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [15]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (215, 15)
y_train:  (215,)
X_test:  (109, 15)
y_test (109,)


In [16]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (215, 15, 1)
X_test:  (109, 15, 1)


In [17]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.4 MB/s eta 0:00:00


In [18]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error


def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_lstm_layers = trial.suggest_int('num_lstm_layers', 1, 3)
    lstm_units = trial.suggest_categorical('lstm_units', [16, 32, 64])
    batch_size = trial.suggest_categorical('batch_size', [5, 10, 16])


    tf.keras.backend.clear_session()

    model = Sequential()
    for i in range(num_lstm_layers):
        if i == 0:
            model.add(LSTM(lstm_units, return_sequences=True, input_shape=(time_step, 1)))
        else:
            model.add(LSTM(lstm_units, return_sequences=True))
    model.add(LSTM(lstm_units))
    model.add(Dense(1))


    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=batch_size, verbose=0)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)


best_params = study.best_params
best_loss = study.best_value
print("Best Hyperparameters: ", best_params)
print("Best Loss: ", best_loss)


[I 2023-12-07 06:48:14,910] A new study created in memory with name: no-name-88e2c5a6-d7db-434b-9685-d0d929ede427
<ipython-input-18-4abf520af7cc>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


4/4 [==============================] - 1s 3ms/step


[I 2023-12-07 06:48:43,762] Trial 0 finished with value: 0.09020447123194475 and parameters: {'learning_rate': 1.3638427440960941e-05, 'num_lstm_layers': 1, 'lstm_units': 16, 'batch_size': 10}. Best is trial 0 with value: 0.09020447123194475.
<ipython-input-18-4abf520af7cc>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


4/4 [==============================] - 1s 4ms/step


[I 2023-12-07 06:49:09,735] Trial 1 finished with value: 0.006994646966152702 and parameters: {'learning_rate': 0.021345399798840488, 'num_lstm_layers': 2, 'lstm_units': 16, 'batch_size': 10}. Best is trial 1 with value: 0.006994646966152702.
<ipython-input-18-4abf520af7cc>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


...

4/4 [==============================] - 1s 5ms/step


[I 2023-12-07 07:20:55,658] Trial 98 finished with value: 0.001471748817177008 and parameters: {'learning_rate': 0.0027178682023217154, 'num_lstm_layers': 1, 'lstm_units': 32, 'batch_size': 16}. Best is trial 37 with value: 0.000657919125360882.
<ipython-input-18-4abf520af7cc>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


4/4 [==============================] - 1s 4ms/step


[I 2023-12-07 07:21:14,791] Trial 99 finished with value: 0.0008264643510949396 and parameters: {'learning_rate': 0.0037548055908541936, 'num_lstm_layers': 1, 'lstm_units': 32, 'batch_size': 5}. Best is trial 37 with value: 0.000657919125360882.


Best Hyperparameters:  {'learning_rate': 0.005262028732848304, 'num_lstm_layers': 1, 'lstm_units': 32, 'batch_size': 5}
Best Loss:  0.000657919125360882


In [19]:
tf.keras.backend.clear_session()
learning_rate=0.005262028732848304

num_lstm_layers = 1
lstm_units = 32
batch_size = 5
num_epochs = 40

model = Sequential()
model.add(LSTM(units=lstm_units, input_shape=(time_step, 1)))

for _ in range(num_lstm_layers - 1):
    model.add(LSTM(units=lstm_units))
model.add(Dense(units=1))
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, verbose=1)
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

Epoch 1/40
43/43 [==============================] - 1s 3ms/step - loss: 0.0140
Epoch 2/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0043
Epoch 3/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 4/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0022
Epoch 5/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0018
Epoch 6/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 7/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 8/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0014
Epoch 9/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0020
Epoch 10/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 11/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0012
Epoch 12/40
43/43 [==============================] - 0s 3ms/step - loss: 0.0010
Epoch 13/40
43/43 [==============================

In [20]:
import pickle

In [21]:
pickle.dump(model, open('highmodel_lstm_fix_banget.sav', 'wb'))

In [22]:
pickle.dump(model, open('highmodel_lstm_fix_banget.h5', 'wb'))

In [23]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

4/4 [==============================] - 0s 3ms/step


((215, 1), (109, 1))

In [24]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

In [25]:
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  10.328660093468203
Train data MSE:  106.68121932640258
Test data MAE:  6.609435822242913
-------------------------------------------------------------------------------------
Test data RMSE:  15.72545151431545
Test data MSE:  247.2898253290861
Test data MAE:  12.478691899010894


In [26]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.976860538056283
Test data explained variance regression score: 0.9624270414359771


In [27]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.974414093948349
Test data R2 score: 0.9322297004859282


In [28]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0010372312392148946
Test data MGD:  0.0010162593488355527
----------------------------------------------------------------------
Train data MPD:  0.3224021915444816
Test data MPD:  0.4975052605801005


In [29]:
look_back=time_step
trainPredictPlot = np.empty_like(argus)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(argus)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(argus)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original price','Train predicted price','Test predicted price'])


plotdf = pd.DataFrame({'Start_date': copy_price['Start_date'],
                       'original_price': copy_price['Argus_High'],
                      'train_predicted': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['Start_date'], y=[plotdf['original_price'],plotdf['train_predicted'],
                                          plotdf['test_predicted']],
              labels={'value':'price','': 'Date'})
fig.update_layout(title_text='Comparision between original price vs predicted price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (356, 1)
Test predicted data:  (356, 1)


In [30]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


In [31]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [32]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(argus[len(argus)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [33]:
lstmdf=argus.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['price'])

fig = px.line(lstmdf,labels={'value': ' price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole  stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes (showgrid=False)
fig.show()

In [34]:
train_result_df = pd.DataFrame({'Original Price': original_ytrain.flatten(), 'Predicted Price': train_predict.flatten()})
test_result_df = pd.DataFrame({'Original Price': original_ytest.flatten(), 'Predicted Price': test_predict.flatten()})

# Display the DataFrames
print("Training Set:")
print(train_result_df)

print("\nTesting Set:")
print(test_result_df)

Training Set:
     Original Price  Predicted Price
0             310.0       308.246613
1             305.0       306.351990
2             301.0       301.022491
3             280.0       297.730743
4             273.0       272.489044
..              ...              ...
210           430.0       432.605743
211           420.0       417.513062
212           420.0       406.186584
213           410.0       409.554657
214           405.0       394.658997

[215 rows x 2 columns]

Testing Set:
     Original Price  Predicted Price
0             375.0       370.159760
1             385.0       366.376556
2             390.0       383.214172
3             395.0       386.303253
4             410.0       393.605225
..              ...              ...
104           491.6       493.602722
105           505.0       482.585632
106           502.0       506.562897
107           497.0       488.400208
108           508.0       487.059326

[109 rows x 2 columns]
